# DIY Multilayer Perceptron
I've written some (questionable) code in [mlp-regressor.py](mlp-regressor.py) to try to implement a multi-layer perceptron. Let's use it to see if we can predict wine quality.

First, you'll need to `pip install ucimlrepo` to get the data-fetching module. The `requirements.txt` file should now be fixed as well (it had some sneaky windows stuff in there).

In [ ]:
# copied from https://archive.ics.uci.edu/dataset/186/wine+quality
from ucimlrepo import fetch_ucirepo 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # lazy splitting
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as numpy arrays)
X = wine_quality.data.features.values
y = wine_quality.data.targets.values

# print out the descriptors
wine_quality.data.original.info()

In [ ]:
# look at the original data
import pandas as pd
pd.plotting.scatter_matrix(wine_quality.data.original, alpha=0.3, figsize=(12, 10))

In [3]:
# append the colour as binary encoded
X = np.column_stack([X, (wine_quality.data.original.color == "red").astype(int)])

# add some jitter to y
y = y + np.random.normal(0, 0.1, y.shape)

In [4]:
# Split into 80/20/20 Train/test/val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=219)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2/0.8, random_state=219)

In [5]:
# calculate norm params on test, excluding colour
mu = X_train[:, :-1].mean(axis=0)
std = X_train[:, :-1].std(axis=0)

X_train[:, :-1] = (X_train[:, :-1] - mu) / std
X_val[:, :-1] = (X_val[:, :-1] - mu) / std
X_test[:, :-1] = (X_test[:, :-1] - mu) / std

## Build and train an MLP
Up to now it's been pretty standard data exploration, preprocessing and splitting. You're welcome to tweak those things, of course.

I've written a class `MLPRegressor` that should be able to train a multi-layer perceptron. It's not very good, but it's a start. You can use it like this:

```python
from mlp_regressor import MLPRegressor
mlp = MLPRegressor(X_train.shape[1])
mlp.add_layer(<number of neurons>, "activation function")
... repeat
print(mlp) # to see a summary of layers

loss = mlp.train(X_train, y_train, step_size, epochs)
plt.plot(loss)
```

It's very inefficient, so don't go too crazy with number of neurons.

In [ ]:
# build a basic MLP
import importlib
import mlp_regressor
# importlib reloads the module from scratch every time you run this cell
# useful if you make any changes to MLPRegressor
importlib.reload(mlp_regressor)
from mlp_regressor import MLPRegressor

In [ ]:
# Try a scatter plot to check performance

## Part 2: Modify the MLP
Right now, the MLP only does whole-batch gradient descent. Modify it to allow training in batches. Does this help the performance?

Try to read through the forward and backward passes to understand how it works. It's entirely possible I've made a mistake somewhere, so don't hesitate to ask if something doesn't make sense.